In [ ]:
# imu_pipeline_for_displacement.py
# Guarda como imu_pipeline_for_displacement.py y ejecútalo en un entorno con numpy, pandas, scipy, matplotlib.

import numpy as np
import pandas as pd
from scipy import signal, optimize
import matplotlib.pyplot as plt
from math import sqrt

# ---------- CONFIG ----------
# Ruta a tu CSV (reemplaza por la ruta real en tu PC)
CSV_PATH = "data/3_IMU/VNIMU_1.csv"
DELIM = ";"  # cambia si tu CSV usa coma ","
FS = 100.0   # frecuencia de muestreo (Hz) — ajusta si tu archivo usa otro valor
# Si conoces factores de escala (LSB -> unidades), colócalos aquí:
ACC_SCALE = 1.0    # ej: 1/16384 si tus datos están en LSB con ±2g
GYRO_SCALE = 1.0   # ej: 1/131 si giros están en LSB (°/s)
MAG_SCALE = 1.0
# ---------- FIN CONFIG ----------

def load_imu_csv(path, delimiter=';'):
    df = pd.read_csv(path, delimiter=delimiter)

    df = pd.read_csv(
    "tu_archivo.csv",
    sep=";",
    decimal=",",
    engine="python"
    )

    # Convertir a float explícitamente
    cols = [
        'UncompMagX','UncompMagY','UncompMagZ',
        'UncompAccX','UncompAccY','UncompAccZ',
        'UncompGyroX','UncompGyroY','UncompGyroZ',
        'Temperature','Pressure'
    ]

    df[cols] = df[cols].apply(lambda x: x.str.replace('+','').str.replace(',', '.').astype(float))

    return df

# Madgwick AHRS (implementación simple)
class MadgwickAHRS:
    def __init__(self, sample_period=1/100, beta=0.1):
        self.sample_period = sample_period
        self.beta = beta
        self.quaternion = np.array([1.0, 0.0, 0.0, 0.0])
    def update_imu(self, gyro, acc):
        q = self.quaternion
        gx, gy, gz = gyro
        ax, ay, az = acc
        norm = sqrt(ax*ax + ay*ay + az*az)
        if norm == 0:
            return
        ax, ay, az = ax/norm, ay/norm, az/norm
        q1, q2, q3, q4 = q
        f1 = 2*(q2*q4 - q1*q3) - ax
        f2 = 2*(q1*q2 + q3*q4) - ay
        f3 = 2*(0.5 - q2*q2 - q3*q3) - az
        J_11or24 = 2*q3
        J_12or23 = 2*q4
        J_13or22 = 2*q1
        J_14or21 = 2*q2
        J_32 = 2*J_14or21
        J_33 = 2*J_11or24
        grad = np.array([
            J_14or21 * f2 - J_11or24 * f1,
            J_12or23 * f1 + J_13or22 * f2 - J_32 * f3,
            J_12or23 * f2 - J_33 * f3 - J_13or22 * f1,
            J_14or21 * f1 + J_11or24 * f2
        ])
        gn = np.linalg.norm(grad)
        if gn != 0: grad = grad / gn
        q_dot = 0.5 * np.array([
            -q2*gx - q3*gy - q4*gz,
             q1*gx + q3*gz - q4*gy,
             q1*gy - q2*gz + q4*gx,
             q1*gz + q2*gy - q3*gx
        ]) - self.beta * grad
        q = q + q_dot * self.sample_period
        self.quaternion = q / np.linalg.norm(q)

def simple_calibration(df, acc_scale=1.0, gyro_scale=1.0, mag_scale=1.0, acc_bias=None, gyro_bias=None, mag_bias=None):
    # Asegura DataFrame
    if isinstance(df, np.ndarray):
        raise ValueError("Pasa un DataFrame, no un numpy array")
    dfc = df.copy()
    if acc_bias is None:
        acc_bias = np.median(dfc[['UncompAccX','UncompAccY','UncompAccZ']].to_numpy(), axis=0)
    if gyro_bias is None:
        gyro_bias = np.median(dfc[['UncompGyroX','UncompGyroY','UncompGyroZ']].to_numpy(), axis=0)
    if mag_bias is None and all(col in dfc.columns for col in ['UncompMagX','UncompMagY','UncompMagZ']):
        mag_bias = np.median(dfc[['UncompMagX','UncompMagY','UncompMagZ']].to_numpy(), axis=0)
    # Aplica bias y escala
    for i,ax in enumerate(['UncompAccX','UncompAccY','UncompAccZ']):
        dfc[ax] = (dfc[ax] - acc_bias[i]) * acc_scale
    for i,gx in enumerate(['UncompGyroX','UncompGyroY','UncompGyroZ']):
        dfc[gx] = (dfc[gx] - gyro_bias[i]) * gyro_scale
    if all(col in dfc.columns for col in ['UncompMagX','UncompMagY','UncompMagZ']) and mag_bias is not None:
        for i,mx in enumerate(['UncompMagX','UncompMagY','UncompMagZ']):
            dfc[mx] = (dfc[mx] - mag_bias[i]) * mag_scale
    return dfc, acc_bias, gyro_bias, mag_bias

def lowpass(data, fs, cutoff=5, order=4):
    nyq = 0.5*fs
    b,a = signal.butter(order, cutoff/nyq, btype='low')
    return signal.filtfilt(b,a,data,axis=0)

def detect_zupt(acc, gyro, fs, acc_thresh=0.05, gyro_thresh=0.5, window=0.05):
    w = int(round(window * fs)); 
    if w < 1: w = 1
    acc_mag = np.linalg.norm(acc, axis=1)
    gyro_mag = np.linalg.norm(gyro, axis=1)
    acc_var = pd.Series(acc_mag).rolling(w, center=True, min_periods=1).std().values
    gyro_var = pd.Series(gyro_mag).rolling(w, center=True, min_periods=1).std().values
    zupt = (acc_var < acc_thresh) & (gyro_var < gyro_thresh)
    return zupt

def integrate_with_zupt(acc_global, zupt, fs):
    dt = 1.0 / fs
    N = acc_global.shape[0]
    vel = np.zeros_like(acc_global)
    pos = np.zeros_like(acc_global)
    for i in range(1, N):
        vel[i] = vel[i-1] + acc_global[i-1]*dt
        pos[i] = pos[i-1] + vel[i-1]*dt + 0.5*acc_global[i-1]*dt*dt
        if zupt[i]:
            vel[i] = np.zeros(3)
    return vel, pos

def imu_pipeline(df, fs=100.0):
    dfc, acc_bias, gyro_bias, mag_bias = simple_calibration(df, acc_scale=ACC_SCALE, gyro_scale=GYRO_SCALE, mag_scale=MAG_SCALE)
    acc = dfc[['UncompAccX','UncompAccY','UncompAccZ']].to_numpy()
    gyro = dfc[['UncompGyroX','UncompGyroY','UncompGyroZ']].to_numpy()
    # Si gyros están en °/s, convertir a rad/s para Madgwick:
    # SI tus giros ya están en rad/s, comenta la siguiente línea
    gyro = gyro * np.pi/180.0
    t = np.arange(len(dfc))/fs
    acc_lp = lowpass(acc, fs, cutoff=5)
    ahrs = MadgwickAHRS(sample_period=1/fs, beta=0.08)
    quats = np.zeros((len(dfc),4))
    for i in range(len(dfc)):
        ahrs.update_imu(gyro[i], acc[i])
        quats[i] = ahrs.quaternion
    acc_global = np.zeros_like(acc)
    for i in range(len(dfc)):
        q1,q2,q3,q4 = quats[i]
        R = np.array([
            [1-2*(q3**2+q4**2),   2*(q2*q3 - q1*q4),   2*(q2*q4 + q1*q3)],
            [2*(q2*q3 + q1*q4),   1-2*(q2**2+q4**2),   2*(q3*q4 - q1*q2)],
            [2*(q2*q4 - q1*q3),   2*(q3*q4 + q1*q2),   1-2*(q2**2+q3**2)]
        ])
        acc_global[i] = R.dot(acc[i])
    # restar gravedad (asume z global hacia arriba)
    acc_global[:,2] -= 9.81
    zupt = detect_zupt(acc_lp, gyro, fs, acc_thresh=0.05, gyro_thresh=0.5, window=0.05)
    vel, pos = integrate_with_zupt(acc_global, zupt, fs)
    return {
        't': t,
        'acc': acc,
        'acc_global': acc_global,
        'gyro': gyro,
        'quats': quats,
        'zupt': zupt,
        'vel': vel,
        'pos': pos
    }

# ---------- USO ----------
if __name__ == "__main__":
    # Cargar (si no tienes el CSV, genera un ejemplo llamando a synthetic o aplica tu CSV)
    try:
        df = load_imu_csv(CSV_PATH, delimiter=DELIM)
        df = df.apply(pd.to_numeric, errors='coerce')
        df = df.dropna()

    except Exception as e:
        print("Error cargando CSV:", e)
        print("Puedes cambiar CSV_PATH a la ruta correcta.")
        raise

    out = imu_pipeline(df, fs=FS)

    df_out = pd.DataFrame({
        't': out['t'],
        'acc_x': out['acc'][:,0],
        'acc_y': out['acc'][:,1],
        'acc_z': out['acc'][:,2],
        'acc_g_x': out['acc_global'][:,0],
        'acc_g_y': out['acc_global'][:,1],
        'acc_g_z': out['acc_global'][:,2],
        'gyro_x': out['gyro'][:,0],
        'gyro_y': out['gyro'][:,1],
        'gyro_z': out['gyro'][:,2],
        'zupt': out['zupt'].astype(int),
        'vel_x': out['vel'][:,0],
        'vel_y': out['vel'][:,1],
        'vel_z': out['vel'][:,2],
        'pos_x': out['pos'][:,0],
        'pos_y': out['pos'][:,1],
        'pos_z': out['pos'][:,2]
    })
    out_path = "imu_pipeline_results.csv"
    df_out.to_csv(out_path, index=False)
    print("Resultados guardados en:", out_path)

    # Plots rápidos
    plt.figure(figsize=(9,6))
    plt.subplot(3,1,1)
    plt.plot(out['t'], df_out['acc_z'])
    plt.title('Acc Z (raw)')
    plt.subplot(3,1,2)
    plt.plot(out['t'], df_out['acc_g_z'])
    plt.title('Acc Z global (gravedad restada)')
    plt.subplot(3,1,3)
    plt.plot(out['t'], df_out['pos_z'])
    plt.title('Pos Z (integrada con ZUPT)')
    plt.tight_layout()
    plt.show()


c:\Users\nicol\anaconda3\envs\AST\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\nicol\anaconda3\envs\AST\Lib\site-packages\numpy\_core\_methods.py:136: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
c:\Users\nicol\anaconda3\envs\AST\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\nicol\anaconda3\envs\AST\Lib\site-packages\numpy\_core\_methods.py:136: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
c:\Users\nicol\anaconda3\envs\AST\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\nicol\anaconda3\envs\AST\Lib\site-packages\numpy\_core\_methods.py:136: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


ValueError: The length of the input vector x must be greater than padlen, which is 15.